In [ ]:
import os
import sys
import csv
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import glob
import math
import cv2
import sklearn
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import models, callbacks
from sklearn.svm import OneClassSVM
from deep_svdd import BaseSVDD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D, Reshape
from tensorflow.keras import optimizers
from tensorflow.keras.applications.efficientnet import decode_predictions
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import MVTecAD_loaddata as mv
import pandas as pd
AUC_list0=[]
AUC_list01=[]
es_cb = keras.callbacks.EarlyStopping(monitor='loss', min_delta=10**(-4), patience=2, mode='auto')
def makelabels(A):
    labels = []
    for i in A:
        if i == 1:
            labels.append(0)
        else:
            labels.append(1)
    return labels

In [2]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# x_train, x_test = train_test_split(X_train, test_size=0.1)
# data_num = X_train.shape[0]
# param_nu = 0.1
# param_C =1/(data_num*param_nu)
# X_train = X_train.astype('float32')
# X_test_error = X_test_error.astype('float32')

# model = Sequential()
# model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
# model.add(Reshape((8,8,1408)))
# model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
# model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
# model.add(UpSampling2D(size=(2, 2)))
# model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
# model.add(UpSampling2D(size=(2, 2)))
# model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
# model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
# model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
# encoder = models.clone_model(model)
# encoder.compile(optimizer='adam', loss='binary_crossentropy')
# encoder.set_weights(model.get_weights())
# for i in range(7):
#     encoder.pop()
# X_tr = encoder.predict(X_train,verbose=0)
# X_ts = encoder.predict(X_test,verbose=0)
# x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
# x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

# svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
# svdd.fit(x_tr)
# svdd.predict(x_ts, y_ts)
# loss = svdd.get_distance(x_ts)
# y_test = makelabels(y_ts)
# auc = sklearn.metrics.roc_auc_score(y_test, loss)
# AUC_list0.append(auc)

# alphas = svdd.alpha
# Param = svdd.C
# m, n = x_tr.shape
# SV = []
# NSV = []
# for i in range(m):
#   if alphas[i] > 0 and alphas[i] <= Param[0]:
#     SV.append(list(X_train[i, :, :, :]))
#   else:
#     NSV.append(list(X_train[i, :, :, :]))
# sv = np.array(SV)
# nsv = np.array(NSV)
# a, b, c, d = sv.shape
# a = a/9.9
# a = int(a)
# model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
# encoder = models.clone_model(model)
# encoder.compile(optimizer='adam', loss='binary_crossentropy')
# encoder.set_weights(model.get_weights())
# for i in range(7):
#     encoder.pop()
# X_tr = encoder.predict(X_train,verbose=0)
# X_ts = encoder.predict(X_test,verbose=0)
# x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
# x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

# svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
# svdd.fit(x_tr)
# loss = svdd.get_distance(x_ts)
# svdd.predict(x_ts, y_ts)
# y_test = makelabels(y_ts)
# auc = sklearn.metrics.roc_auc_score(y_test, loss)
# AUC_list01.append(auc)

In [3]:
%%time
for num in range(1):
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
#     param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
#     model.add(Reshape((8,8,1408)))
#     model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
#     model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
#     model.add(UpSampling2D(size=(2, 2)))
#     model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
#     model.add(UpSampling2D(size=(2, 2)))
#     model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
#     model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
#     encoder = models.clone_model(model)
#     encoder.compile(optimizer='adam', loss='binary_crossentropy')
#     encoder.set_weights(model.get_weights())
#     for i in range(7):
#         encoder.pop()
    X_tr = model.predict(X_train,verbose=0)
    X_ts = model.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)



*** Fitting of the SVDD model is completed. ***

running time         = 2.1557 seconds
kernel function      = rbf
iterations           = 14
radius               = 0.2791
number of samples    = 209
number of features   = 90112
number of SVs        = 24
ratio of SVs         = 11.4833 %
accuracy             = 95.6938 %




*** Prediction of the provided data is completed. ***

running time         = 0.8335 seconds
number of samples    = 83
number of alarm      = 64
accuracy             = 96.3855 %


Epoch 1/20
12/12 [==============================] - 45s 3s/step - loss: 0.1288 - accuracy: 0.4142 - val_loss: 0.1300 - val_accuracy: 0.4157
Epoch 2/20
12/12 [==============================] - 36s 3s/step - loss: 0.1081 - accuracy: 0.3864 - val_loss: 0.1227 - val_accuracy: 0.3923
Epoch 3/20
12/12 [==============================] - 40s 3s/step - loss: 0.1006 - accuracy: 0.4085 - val_loss: 0.1225 - val_accuracy: 0.4015
Epoch 4/20
12/12 [==============================] - 36s 3s/step - loss: 0.09

12/12 [==============================] - 37s 3s/step - loss: 0.0966 - accuracy: 0.3381 - val_loss: 0.1156 - val_accuracy: 0.3357
Epoch 6/20
12/12 [==============================] - 37s 3s/step - loss: 0.0953 - accuracy: 0.3539 - val_loss: 0.1156 - val_accuracy: 0.3307
Epoch 7/20
12/12 [==============================] - 37s 3s/step - loss: 0.0941 - accuracy: 0.3539 - val_loss: 0.1147 - val_accuracy: 0.3275
Epoch 8/20
12/12 [==============================] - 37s 3s/step - loss: 0.0929 - accuracy: 0.3675 - val_loss: 0.1110 - val_accuracy: 0.3365
Epoch 9/20
12/12 [==============================] - 37s 3s/step - loss: 0.0915 - accuracy: 0.3753 - val_loss: 0.1076 - val_accuracy: 0.3520
Epoch 10/20
12/12 [==============================] - 37s 3s/step - loss: 0.0892 - accuracy: 0.4171 - val_loss: 0.1032 - val_accuracy: 0.3946
Epoch 11/20
12/12 [==============================] - 37s 3s/step - loss: 0.0788 - accuracy: 0.4714 - val_loss: 0.0859 - val_accuracy: 0.4263
Epoch 12/20
12/12 [==========

In [4]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)



*** Fitting of the SVDD model is completed. ***

running time         = 2.1810 seconds
kernel function      = rbf
iterations           = 11
radius               = 0.6088
number of samples    = 224
number of features   = 90112
number of SVs        = 38
ratio of SVs         = 16.9643 %
accuracy             = 95.9821 %




*** Prediction of the provided data is completed. ***

running time         = 1.3036 seconds
number of samples    = 140
number of alarm      = 63
accuracy             = 76.4286 %


Epoch 1/20
13/13 [==============================] - 59s 4s/step - loss: 0.0537 - accuracy: 0.3962 - val_loss: 0.0523 - val_accuracy: 0.5656
Epoch 2/20
13/13 [==============================] - 48s 4s/step - loss: 0.0480 - accuracy: 0.6475 - val_loss: 0.0468 - val_accuracy: 0.7016
Epoch 3/20
13/13 [==============================] - 48s 4s/step - loss: 0.0425 - accuracy: 0.7377 - val_loss: 0.0429 - val_accuracy: 0.7803
Epoch 4/20
13/13 [==============================] - 47s 4s/step - loss: 0.0

13/13 [==============================] - 52s 4s/step - loss: 0.0341 - accuracy: 0.7610 - val_loss: 0.0379 - val_accuracy: 0.7775
Epoch 6/20
13/13 [==============================] - 52s 4s/step - loss: 0.0324 - accuracy: 0.7862 - val_loss: 0.0382 - val_accuracy: 0.7985
Epoch 7/20
13/13 [==============================] - 52s 4s/step - loss: 0.0310 - accuracy: 0.7986 - val_loss: 0.0369 - val_accuracy: 0.8048
Epoch 8/20
13/13 [==============================] - 52s 4s/step - loss: 0.0297 - accuracy: 0.8061 - val_loss: 0.0365 - val_accuracy: 0.8107
Epoch 9/20
13/13 [==============================] - 52s 4s/step - loss: 0.0280 - accuracy: 0.8166 - val_loss: 0.0342 - val_accuracy: 0.8139
Epoch 10/20
13/13 [==============================] - 53s 4s/step - loss: 0.0248 - accuracy: 0.8196 - val_loss: 0.0293 - val_accuracy: 0.8115
Epoch 11/20
13/13 [==============================] - 52s 4s/step - loss: 0.0208 - accuracy: 0.8155 - val_loss: 0.0255 - val_accuracy: 0.8158
Epoch 12/20
13/13 [==========

In [5]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)



*** Fitting of the SVDD model is completed. ***

running time         = 2.2146 seconds
kernel function      = rbf
iterations           = 13
radius               = 0.3489
number of samples    = 219
number of features   = 90112
number of SVs        = 28
ratio of SVs         = 12.7854 %
accuracy             = 95.8904 %




*** Prediction of the provided data is completed. ***

running time         = 1.2396 seconds
number of samples    = 132
number of alarm      = 47
accuracy             = 46.9697 %


Epoch 1/20
14/14 [==============================] - 57s 4s/step - loss: 0.0932 - accuracy: 0.2989 - val_loss: 0.0916 - val_accuracy: 0.3346
Epoch 2/20
14/14 [==============================] - 48s 4s/step - loss: 0.0821 - accuracy: 0.3292 - val_loss: 0.0857 - val_accuracy: 0.3381
Epoch 3/20
14/14 [==============================] - 48s 4s/step - loss: 0.0647 - accuracy: 0.3464 - val_loss: 0.0668 - val_accuracy: 0.3308
Epoch 4/20
14/14 [==============================] - 48s 4s/step - loss: 0.0

14/14 [==============================] - 51s 4s/step - loss: 0.0229 - accuracy: 0.4641 - val_loss: 0.0565 - val_accuracy: 0.4188
Epoch 7/20
14/14 [==============================] - 51s 4s/step - loss: 0.0226 - accuracy: 0.4696 - val_loss: 0.0579 - val_accuracy: 0.4251
Epoch 8/20
14/14 [==============================] - 51s 4s/step - loss: 0.0222 - accuracy: 0.4759 - val_loss: 0.0571 - val_accuracy: 0.4357
Epoch 9/20
14/14 [==============================] - 51s 4s/step - loss: 0.0219 - accuracy: 0.4928 - val_loss: 0.0582 - val_accuracy: 0.4341
Epoch 10/20
14/14 [==============================] - 51s 4s/step - loss: 0.0220 - accuracy: 0.4873 - val_loss: 0.0604 - val_accuracy: 0.4214
Epoch 11/20
14/14 [==============================] - 51s 4s/step - loss: 0.0218 - accuracy: 0.4969 - val_loss: 0.0626 - val_accuracy: 0.4376


*** Fitting of the SVDD model is completed. ***

running time         = 2.2026 seconds
kernel function      = rbf
iterations           = 11
radius               = 0.80

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)



*** Fitting of the SVDD model is completed. ***

running time         = 2.7962 seconds
kernel function      = rbf
iterations           = 8
radius               = 0.1431
number of samples    = 280
number of features   = 90112
number of SVs        = 36
ratio of SVs         = 12.8571 %
accuracy             = 95.3571 %




*** Prediction of the provided data is completed. ***

running time         = 1.3841 seconds
number of samples    = 136
number of alarm      = 107
accuracy             = 81.6176 %


Epoch 1/20
12/12 [==============================] - 75s 6s/step - loss: 0.0386 - accuracy: 0.1974 - val_loss: 0.0374 - val_accuracy: 0.1687
Epoch 2/20
12/12 [==============================] - 57s 5s/step - loss: 0.0356 - accuracy: 0.1630 - val_loss: 0.0338 - val_accuracy: 0.1368
Epoch 3/20
12/12 [==============================] - 57s 5s/step - loss: 0.0288 - accuracy: 0.1542 - val_loss: 0.0276 - val_accuracy: 0.1565
Epoch 4/20
12/12 [==============================] - 57s 5s/step - loss: 0.0

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = sv.shape
    a = a/9.9
    a = int(a)
    model.fit(sv, sv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
a = np.array(AUC_list)
a= a.reshape(15,2)
df = pd.DataFrame(a)
df.to_csv('EN_SVDD_sv01.csv')

In [ ]:
import os
import sys
import csv
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import glob
import math
import cv2
import sklearn
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import models, callbacks
from sklearn.svm import OneClassSVM
from deep_svdd import BaseSVDD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D, Reshape
from tensorflow.keras import optimizers
from tensorflow.keras.applications.efficientnet import decode_predictions
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import MVTecAD_loaddata as mv
import pandas as pd
AUC_list0=[]
AUC_list01=[]
es_cb = keras.callbacks.EarlyStopping(monitor='loss', min_delta=10**(-4), patience=2, mode='auto')
def makelabels(A):
    labels = []
    for i in A:
        if i == 1:
            labels.append(0)
        else:
            labels.append(1)
    return labels

In [ ]:
%%time
for num in range(5):
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
%%time
for num in range(5):
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
#     X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
    X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
    x_train, x_test = train_test_split(X_train, test_size=0.1)
    data_num = X_train.shape[0]
    param_nu = 0.1
    param_C =1/(data_num*param_nu)
    X_train = X_train.astype('float32')
    X_test_error = X_test_error.astype('float32')

    model = Sequential()
    model.add(EfficientNetB2(include_top=False,weights='imagenet', input_shape=(256,256,3)))
    model.add(Reshape((8,8,1408)))
    model.add(Conv2D(256, kernel_size=(3, 3),activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3, 3),activation='relu', padding='same',strides=2))
    model.add(Conv2D(3, kernel_size=(3, 3),activation='sigmoid', padding='same'))
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    svdd.predict(x_ts, y_ts)
    loss = svdd.get_distance(x_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list0.append(auc)

    alphas = svdd.alpha
    Param = svdd.C
    m, n = x_tr.shape
    SV = []
    NSV = []
    for i in range(m):
      if alphas[i] > 0 and alphas[i] <= Param[0]:
        SV.append(list(X_train[i, :, :, :]))
      else:
        NSV.append(list(X_train[i, :, :, :]))
    sv = np.array(SV)
    nsv = np.array(NSV)
    a, b, c, d = nsv.shape
    a = a/9.9
    a = int(a)
    model.fit(nsv, nsv,epochs=20,batch_size=a,shuffle=True,validation_data=(X_train, X_train),callbacks=[es_cb])
    encoder = models.clone_model(model)
    encoder.compile(optimizer='adam', loss='binary_crossentropy')
    encoder.set_weights(model.get_weights())
    for i in range(7):
        encoder.pop()
    X_tr = encoder.predict(X_train,verbose=0)
    X_ts = encoder.predict(X_test,verbose=0)
    x_tr = np.reshape(X_tr,(X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]*X_tr.shape[3]))
    x_ts = np.reshape(X_ts,(X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]*X_ts.shape[3]))

    svdd = BaseSVDD(C=param_C, gamma='scale', kernel='rbf', display='on')
    svdd.fit(x_tr)
    loss = svdd.get_distance(x_ts)
    svdd.predict(x_ts, y_ts)
    y_test = makelabels(y_ts)
    auc = sklearn.metrics.roc_auc_score(y_test, loss)
    AUC_list01.append(auc)

In [ ]:
a = np.array(AUC_list)
a= a.reshape(15,2)
df = pd.DataFrame(a)
df.to_csv('EN_SVDD_nsv01.csv')

In [ ]:
len(AUC_list01)